In [26]:
import pyscf

import numpy as np
import scipy as sp

import math
import itertools

def rudimentary_fci_matrix(h1e, h2e, norb, nelec):

    # We need to generate all possible determinants.
    # Includes both alpha and beta electrons...

    # We can use the itertools library to generate combinations of orbitals.
    # We are not restricting the possible spin yet.

    determinants = list(itertools.combinations(range(2*norb), nelec))
    # This will give us all possible combinations of spinorbitals for the given number of electrons.

    dim = len(determinants)
    # This is the dimension of the FCI space.

    def count_diff(det1, det2):
        return len(det1) - len(list(set(det1).intersection(set(det2))))

    # slater condon rules:
    def slater_condon_rule(det1, det2):

        one_electron_energy = 0.0
        two_electron_energy = 0.0
        # We will calculate the one-electron and two-electron contributions separately.

        diff = count_diff(det1, det2)

        if diff > 2:
            return 0.0

        elif diff == 0:
            # then the two determinants are the same; diagonal element.
            for i in det1:
                one_electron_energy += h1e[i % norb, i % norb]
            for i in det1:
                for j in det1:
                    two_electron_energy += 0.5*h2e[i%norb, i%norb, j%norb, j%norb]
                    if i < norb and j < norb:
                        two_electron_energy -= 0.5*h2e[i%norb, j%norb, j%norb, i%norb]
                    elif i >= norb and j >= norb:
                        two_electron_energy -= 0.5*h2e[i%norb, j%norb, j%norb, i%norb]
            return one_electron_energy + two_electron_energy
        
        elif diff == 1:
            diff_det1 = []
            diff_det2 = []
            nswaps1 = 0
            nswaps2 = 0
            # which elements are different?
            for i in range(nelec):
                # if i not in det2, then need to permute corresponding element to the end.
                if det1[i] not in det2:
                    diff_det1.append(det1[i])
                    nswaps1 += nelec - i - 1
                if det2[i] not in det1:
                    diff_det2.append(det2[i])
                    nswaps2 += nelec - i - 1
            
            nswaps = nswaps1 + nswaps2
            # now we have the two different elements.
            if diff_det1[0] < norb and diff_det2[0] >= norb:
                return 0.0
            elif diff_det1[0] >= norb and diff_det2[0] < norb:
                return 0.0
            else:
            # We can calculate the one-electron contribution.
                one_electron_energy += h1e[diff_det1[0] % norb, diff_det2[0] % norb]
                # and the two-electron contribution.
                for i in range(nelec):
                    if det1[i] != diff_det1[0]:
                        two_electron_energy += h2e[diff_det1[0] % norb, diff_det2[0] % norb, det1[i] % norb, det1[i] % norb]
                        if det1[i] < norb and diff_det1[0] < norb:
                            two_electron_energy -= h2e[diff_det1[0] % norb, det1[i] % norb, det1[i] % norb, diff_det2[0] % norb]
                        elif det1[i] >= norb and diff_det1[0] >= norb:
                            two_electron_energy -= h2e[diff_det1[0] % norb, det1[i] % norb, det1[i] % norb, diff_det2[0] % norb]
            one_electron_energy *= (-1)**nswaps
            two_electron_energy *= (-1)**nswaps
            return one_electron_energy + two_electron_energy

        elif diff == 2:
            diff_det1 = []
            diff_det2 = []
            nswaps1 = 0
            nswaps2 = 0

            for i in range(nelec):
                if det1[i] not in det2:
                    diff_det1.append(det1[i])
                    nswaps1 += nelec - i - 1
                if det2[i] not in det1:
                    diff_det2.append(det2[i])
                    nswaps2 += nelec - i - 1
            
            nswaps = nswaps1 + nswaps2

            if diff_det1[0] < norb and diff_det2[0] >= norb:
                return 0.0
            elif diff_det1[0] >= norb and diff_det2[0] < norb:
                return 0.0
            elif diff_det1[1] < norb and diff_det2[1] >= norb:
                return 0.0
            elif diff_det1[1] >= norb and diff_det2[1] < norb:
                return 0.0
            else:
                # only contribution is going to be from the two-electron term.
                two_electron_energy += h2e[diff_det1[0] % norb, diff_det2[0] % norb, diff_det1[1] % norb, diff_det2[1] % norb]
                if diff_det1[1] < norb and diff_det2[1] < norb:
                    two_electron_energy -= h2e[diff_det1[0] % norb, diff_det2[1] % norb, diff_det2[0] % norb, diff_det1[1] % norb]
                elif diff_det1[0] >= norb and diff_det2[0] >= norb:
                    two_electron_energy -= h2e[diff_det1[0] % norb, diff_det2[1] % norb, diff_det2[0] % norb, diff_det1[1] % norb]
            one_electron_energy = 0.0
            two_electron_energy *= (-1)**nswaps
            return one_electron_energy + two_electron_energy
    
    # Now we can construct the Hamiltonian matrix.
    hamiltonian = np.zeros((dim, dim))

    for i in range(dim):
        for j in range(dim):
            hamiltonian[i, j] = slater_condon_rule(determinants[i], determinants[j])

    # symmetrize
    hamiltonian = 0.5 * (hamiltonian + hamiltonian.T)
    
    return hamiltonian

In [27]:
hamiltonian_fci = rudimentary_fci_matrix(
    h1e=np.load("h1e.npy"),
    h2e=np.load("h2e.npy"),
    norb=6,
    nelec=6
)

In [29]:
def rudimentaryDavidson(matrix):

    # create a guess matrix
    dimension = len(matrix)
    guessV1 = np.zeros(dimension)
    guessV2 = np.zeros(dimension)
    for i in range(dimension):
        guessV1[i] = np.random.rand()
        guessV2[i] = np.random.rand()
    guessV1 /= np.linalg.norm(guessV1)
    guessV2 /= np.linalg.norm(guessV2)
    # Gram-Schmidt orthogonalization
    guessV2 -= np.dot(guessV2.T, guessV1) * guessV1
    guessV2 /= np.linalg.norm(guessV2)
    guessV1 = np.reshape(guessV1, (dimension, 1))
    guessV2 = np.reshape(guessV2, (dimension, 1))
    Vectors = np.hstack((guessV1, guessV2))
    norm = 1

    eigenvector = []
    eigenvalue = 0

    while norm > 0.00001:
        # project the FCI matrix onto the guess basis
        projection = np.dot(Vectors.T, np.dot(matrix, Vectors))
        # diagonalize the projection
        theta, s = np.linalg.eigh(projection)
        # compute the residue vector
        r = np.dot((np.dot(matrix,Vectors)-theta[0]*Vectors),s[:,0])
        # diagonal preconditioning
        q = -r.T / (np.diag(matrix) - theta[0] + 1E-18)
        q /= np.linalg.norm(q)
        q = np.reshape(q, (dimension, 1))
        eigenvector = np.dot(Vectors, s[:,0])
        eigenvalue = theta[0]

        Vectors = np.hstack((Vectors, q))
        # QR to orthogonalize
        Vectors = np.linalg.qr(Vectors)[0]

        norm = np.linalg.norm(r)
        
    return eigenvector, eigenvalue

# But I should try implementing my own methods...


In [30]:
vec, val = rudimentaryDavidson(hamiltonian_fci)
print(val)

-7.839908014887425


In [87]:
from copy import deepcopy

import pyscf

import numpy as np
import scipy as sp

import math
import itertools


def singlet_direct_ci_method(h1e, h2e, norb, nelec):
    nalpha = nelec//2
    nbeta = nelec//2

    possiblestrings = list(itertools.combinations(range(norb), nalpha))
    possiblestringsarray = [list(x) for x in possiblestrings]
    numb_alpha_dets = len(possiblestrings)

    def nswaps(det):
        swaps = 0
        thisdet = deepcopy(det)
        sorteddet = sorted(thisdet)
        issorted = False
        while not issorted:
            if thisdet == sorteddet:
                issorted = True
            for i in range(len(thisdet)-1):
                if thisdet[i] > thisdet[i+1]:
                    deti = thisdet[i]
                    detnext = thisdet[i+1]
                    thisdet[i] = detnext
                    thisdet[i+1] = deti
                    swaps += 1
        return swaps

    def f(input_index, mu, nu):
        det = deepcopy(possiblestringsarray[input_index])
        if nu not in det:
            return None
        else:
            det.remove(nu)
            if mu in det:
                return None
            else:
                det.append(mu)
                det = sorted(det)
                det = tuple(det)
                output_index = possiblestrings.index(det)
                return output_index
    
    def sigma(input_index, mu, nu):
        det = deepcopy(possiblestringsarray[input_index])
        if nu not in det:
            return None
        else:
            det.remove(nu)
            if mu in det:
                return None
            else:
                det.append(mu)
                swaps = nswaps(det)
                return swaps
    


    # This is defined for spin-free; but now we need separate for alpha vs. for beta.
        
    def big_sigma(input_index, mu, nu):
        alpha_index =  input_index % numb_alpha_dets
        beta_index = (input_index - alpha_index) // numb_alpha_dets

        if mu < norb and nu < norb:
            #we act only on alpha
            return sigma(alpha_index, mu, nu)

        elif mu >= norb and nu >= norb:
            # do something
            return sigma(beta_index, mu % norb, nu % norb)

        else:
            return None


    def big_f(input_index, mu, nu):
        alpha_index = input_index % numb_alpha_dets
        beta_index = (input_index - alpha_index) // numb_alpha_dets

        if mu < norb and nu < norb:
            # we act only on alpha
            new_alpha_index = f(alpha_index, mu, nu)
            if new_alpha_index is None:
                return None
            else:
                output_index = beta_index * numb_alpha_dets + new_alpha_index
                return output_index

        elif mu >= norb and nu >= norb:
            # we act only on beta

            new_beta_index = f(beta_index, mu % norb, nu % norb)
            if new_beta_index is None:
                return None
            else:
                output_index = new_beta_index * numb_alpha_dets + alpha_index
                return output_index

        else:
            return None


    # now what happens when we act H onto every SD?

    hamiltonian = np.zeros((len(possiblestrings)**2, len(possiblestrings)**2))


    gijjk = np.einsum('ijjk->ik', h2e)

    def one_electron_term():
        for i in range(numb_alpha_dets**2):
            for j in range(norb*2):
                for k in range(norb*2):
                    if big_sigma(i, j, k) is not None:
                        hamiltonian[i][big_f(i,j,k)] += (h1e[j%norb, k%norb] - 0.5*gijjk[j%norb, k%norb]) * (-1)**(big_sigma(i,j,k))
    

    def two_electron_coulomb():
        for term in range(numb_alpha_dets**2):
            for i in range(norb*2):
                for j in range(norb*2):
                    f_inter = big_f(term, i, j)
                    sig_inter = big_sigma(term, i, j)
                    if sig_inter is not None:
                        for k in range(norb*2):
                            for l in range(norb*2):

                                f_final = big_f(f_inter, k, l)
                                sig_final = big_sigma(f_inter, k, l)

                                if sig_final is not None:
                                    hamiltonian[term][f_final] += 0.5*h2e[i%norb, j%norb, k%norb, l%norb] * (-1)**(sig_inter + sig_final)

    one_electron_term()
    two_electron_coulomb()

    return hamiltonian   

ham_dir = singlet_direct_ci_method(
    h1e=np.load("h1e.npy"),
    h2e=np.load("h2e.npy"),
    norb=6,
    nelec=6
)

eigvals_direct = np.linalg.eigh(ham_dir)[0]
print(eigvals_direct)

[-4.34321743 -4.33031592 -4.22504842 -4.20932702 -4.19708957 -4.17335095
 -4.15183897 -4.15027799 -4.12785092 -4.11834982 -4.11429289 -4.0998278
 -4.0832055  -4.06531337 -4.05604433 -4.04518844 -4.03962505 -4.03317915
 -4.01923198 -4.01733628 -4.00517404 -4.00046274 -3.98638762 -3.97868783
 -3.97820893 -3.97095303 -3.95278117 -3.9501999  -3.93680307 -3.92560642
 -3.9124052  -3.90980824 -3.9036209  -3.88895061 -3.88426553 -3.88423608
 -3.87676485 -3.86345606 -3.85966158 -3.84976798 -3.84379892 -3.84102559
 -3.83202906 -3.81995069 -3.81632744 -3.81034228 -3.80879177 -3.79743885
 -3.78496792 -3.78049181 -3.77811735 -3.77461328 -3.76762894 -3.76537135
 -3.75883571 -3.75452538 -3.74705589 -3.73296572 -3.72925838 -3.7238987
 -3.71885586 -3.70382288 -3.70301187 -3.70098267 -3.69615038 -3.68946627
 -3.68572176 -3.6826509  -3.67204648 -3.66372501 -3.6525067  -3.64464246
 -3.64336514 -3.63903302 -3.63575527 -3.63187214 -3.62550834 -3.61972986
 -3.61473005 -3.60735444 -3.60634393 -3.60019387 -3.5

In [81]:
eigvals_full = np.linalg.eigh(hamiltonian_fci)[0]

In [84]:
for i in eigvals_direct:
    print(i)

-4.343217427739944
-4.330315922149541
-4.225048423537944
-4.2093270207659454
-4.197089569164095
-4.173350952821295
-4.151838968582591
-4.150277989409962
-4.127850918991255
-4.1183498199707
-4.114292891915824
-4.099827803356433
-4.083205504434164
-4.065313373228127
-4.056044331759663
-4.045188440313191
-4.039625052380704
-4.0331791497068545
-4.019231984937942
-4.017336277393672
-4.005174038859736
-4.000462738505268
-3.9863876240202663
-3.9786878268065027
-3.978208928606541
-3.9709530339310692
-3.952781166178544
-3.9501998995627914
-3.9368030655418456
-3.925606421315043
-3.912405202974466
-3.90980823552993
-3.903620898525793
-3.8889506132554903
-3.8842655299752025
-3.8842360846659285
-3.876764846400373
-3.863456055203286
-3.8596615772772287
-3.849767983476035
-3.8437989182987775
-3.8410255947882663
-3.8320290614150876
-3.8199506949467787
-3.816327442981198
-3.8103422824318773
-3.8087917673127283
-3.797438854851148
-3.784967918400921
-3.7804918061216974
-3.7781173494023474
-3.774613284358

In [78]:
h2e = np.load("h2e.npy")

for i in range(np.shape(h2e)[0]):
    for j in range(np.shape(h2e)[0]):
        for k in range(np.shape(h2e)[0]):
            for l in range(np.shape(h2e)[0]):
                if not (np.allclose(h2e[i,j,k,l],h2e[j,i,k,l])):
                    print("Error")




In [66]:
ham_dir = singlet_direct_ci_method(
    h1e=np.load("h1e.npy"),
    h2e=np.load("h2e.npy"),
    norb=6,
    nelec=6
)

In [67]:
np.linalg.eigh(ham_dir)[0]

array([-4.34321743, -4.33031592, -4.22504842, -4.20932702, -4.19708957,
       -4.17335095, -4.15183897, -4.15027799, -4.12785092, -4.11834982,
       -4.11429289, -4.0998278 , -4.0832055 , -4.06531337, -4.05604433,
       -4.04518844, -4.03962505, -4.03317915, -4.01923198, -4.01733628,
       -4.00517404, -4.00046274, -3.98638762, -3.97868783, -3.97820893,
       -3.97095303, -3.95278117, -3.9501999 , -3.93680307, -3.92560642,
       -3.9124052 , -3.90980824, -3.9036209 , -3.88895061, -3.88426553,
       -3.88423608, -3.87676485, -3.86345606, -3.85966158, -3.84976798,
       -3.84379892, -3.84102559, -3.83202906, -3.81995069, -3.81632744,
       -3.81034228, -3.80879177, -3.79743885, -3.78496792, -3.78049181,
       -3.77811735, -3.77461328, -3.76762894, -3.76537135, -3.75883571,
       -3.75452538, -3.74705589, -3.73296572, -3.72925838, -3.7238987 ,
       -3.71885586, -3.70382288, -3.70301187, -3.70098267, -3.69615038,
       -3.68946627, -3.68572176, -3.6826509 , -3.67204648, -3.66

In [17]:
possiblestrings = list(itertools.combinations(range(6), 3))
print(len(possiblestrings))


20


In [8]:
print(list(itertools.combinations(range(6),3)))
thislist = list(itertools.combinations(range(6),3))
thisarray = [list(x) for x in thislist]
print(thisarray)

det5 = thislist[5]
print(det5)
list5 = list(det5)
print(list5)
list5.remove(4)
print(list5)

newdet5 = thisarray[5]
print(newdet5)
newdet5.remove(4)
print(newdet5)
newdet5.append(1)
print(newdet5)
newdet5 = sorted(newdet5)
print(newdet5)

# need a method to reorder newdet5
# kendall-tau distance = bubble-sort distance.

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (0, 4, 5), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 4, 5)]
[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 2, 3], [0, 2, 4], [0, 2, 5], [0, 3, 4], [0, 3, 5], [0, 4, 5], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5]]
(0, 2, 4)
[0, 2, 4]
[0, 2]
[0, 2, 4]
[0, 2]
[0, 2, 1]
[0, 1, 2]


In [11]:
def nswaps(det):
    swaps = 0
    thisdet = deepcopy(det)
    sorteddet = sorted(thisdet)
    issorted = False
    while not issorted:
        if thisdet == sorteddet:
            issorted = True
        for i in range(len(thisdet)-1):
            if thisdet[i] > thisdet[i+1]:
                deti = thisdet[i]
                detnext = thisdet[i+1]
                thisdet[i] = detnext
                thisdet[i+1] = deti
                swaps += 1
    return swaps

In [88]:
# debugging
import pyscf

import numpy as np
import scipy as sp

import math
import itertools

def debugging_rudimentary_fci_matrix(h1e, h2e, norb, nelec):

    # We need to generate all possible determinants.
    # Includes both alpha and beta electrons...

    # We can use the itertools library to generate combinations of orbitals.
    # We are not restricting the possible spin yet.

    determinants = list(itertools.combinations(range(2*norb), nelec))
    # This will give us all possible combinations of spinorbitals for the given number of electrons.

    dim = len(determinants)
    # This is the dimension of the FCI space.

    def count_diff(det1, det2):
        return len(det1) - len(list(set(det1).intersection(set(det2))))

    # slater condon rules:
    def slater_condon_rule(det1, det2):

        one_electron_energy = 0.0
        two_electron_energy = 0.0
        # We will calculate the one-electron and two-electron contributions separately.

        diff = count_diff(det1, det2)

        if diff > 2:
            return 0.0

        elif diff == 0:
            # then the two determinants are the same; diagonal element.
            for i in det1:
                one_electron_energy += h1e[i % norb, i % norb]
            for i in det1:
                for j in det1:
                    # two_electron_energy += 0.5*h2e[i%norb, i%norb, j%norb, j%norb]
                    two_electron_energy += 0.0
                    if i < norb and j < norb:
                        # two_electron_energy -= 0.5*h2e[i%norb, j%norb, j%norb, i%norb]
                        two_electron_energy -= 0.0
                    elif i >= norb and j >= norb:
                        # two_electron_energy -= 0.5*h2e[i%norb, j%norb, j%norb, i%norb]
                        two_electron_energy -= 0.0

            return one_electron_energy + two_electron_energy
        
        elif diff == 1:
            diff_det1 = []
            diff_det2 = []
            nswaps1 = 0
            nswaps2 = 0
            # which elements are different?
            for i in range(nelec):
                # if i not in det2, then need to permute corresponding element to the end.
                if det1[i] not in det2:
                    diff_det1.append(det1[i])
                    nswaps1 += nelec - i - 1
                if det2[i] not in det1:
                    diff_det2.append(det2[i])
                    nswaps2 += nelec - i - 1
            
            nswaps = nswaps1 + nswaps2
            # now we have the two different elements.
            if diff_det1[0] < norb and diff_det2[0] >= norb:
                return 0.0
            elif diff_det1[0] >= norb and diff_det2[0] < norb:
                return 0.0
            else:
            # We can calculate the one-electron contribution.
                one_electron_energy += h1e[diff_det1[0] % norb, diff_det2[0] % norb]
                # and the two-electron contribution.
                for i in range(nelec):
                    if det1[i] != diff_det1[0]:
                        # two_electron_energy += h2e[diff_det1[0] % norb, diff_det2[0] % norb, det1[i] % norb, det1[i] % norb]
                        two_electron_energy += 0.0

                        if det1[i] < norb and diff_det1[0] < norb:
                            # two_electron_energy -= h2e[diff_det1[0] % norb, det1[i] % norb, det1[i] % norb, diff_det2[0] % norb]
                            two_electron_energy += 0.0

                        elif det1[i] >= norb and diff_det1[0] >= norb:
                            # two_electron_energy -= h2e[diff_det1[0] % norb, det1[i] % norb, det1[i] % norb, diff_det2[0] % norb]
                            two_electron_energy += 0.0

            one_electron_energy *= (-1)**nswaps
            two_electron_energy *= (-1)**nswaps
            return one_electron_energy + two_electron_energy

        elif diff == 2:
            diff_det1 = []
            diff_det2 = []
            nswaps1 = 0
            nswaps2 = 0

            for i in range(nelec):
                if det1[i] not in det2:
                    diff_det1.append(det1[i])
                    nswaps1 += nelec - i - 1
                if det2[i] not in det1:
                    diff_det2.append(det2[i])
                    nswaps2 += nelec - i - 1
            
            nswaps = nswaps1 + nswaps2

            if diff_det1[0] < norb and diff_det2[0] >= norb:
                return 0.0
            elif diff_det1[0] >= norb and diff_det2[0] < norb:
                return 0.0
            elif diff_det1[1] < norb and diff_det2[1] >= norb:
                return 0.0
            elif diff_det1[1] >= norb and diff_det2[1] < norb:
                return 0.0
            else:
                # only contribution is going to be from the two-electron term.
                # two_electron_energy += h2e[diff_det1[0] % norb, diff_det2[0] % norb, diff_det1[1] % norb, diff_det2[1] % norb]
                two_electron_energy += 0.0

                if diff_det1[1] < norb and diff_det2[1] < norb:
                    # two_electron_energy -= h2e[diff_det1[0] % norb, diff_det2[1] % norb, diff_det2[0] % norb, diff_det1[1] % norb]
                    two_electron_energy += 0.0

                elif diff_det1[0] >= norb and diff_det2[0] >= norb:
                    # two_electron_energy -= h2e[diff_det1[0] % norb, diff_det2[1] % norb, diff_det2[0] % norb, diff_det1[1] % norb]
                    two_electron_energy += 0.0

            one_electron_energy = 0.0
            two_electron_energy *= (-1)**nswaps
            return one_electron_energy + two_electron_energy
    
    # Now we can construct the Hamiltonian matrix.
    hamiltonian = np.zeros((dim, dim))

    for i in range(dim):
        for j in range(dim):
            hamiltonian[i, j] = slater_condon_rule(determinants[i], determinants[j])

    # symmetrize
    hamiltonian = 0.5 * (hamiltonian + hamiltonian.T)
    
    return hamiltonian

In [99]:
    def nswaps(det):
        swaps = 0
        thisdet = deepcopy(det)
        sorteddet = sorted(thisdet)
        issorted = False
        while not issorted:
            if thisdet == sorteddet:
                issorted = True
            for i in range(len(thisdet)-1):
                if thisdet[i] > thisdet[i+1]:
                    deti = thisdet[i]
                    detnext = thisdet[i+1]
                    thisdet[i] = detnext
                    thisdet[i+1] = deti
                    swaps += 1
        return swaps

In [101]:
nswaps([0, 1, 12, 3, 4, 5])

3

In [98]:
from copy import deepcopy

import pyscf

import numpy as np
import scipy as sp

import math
import itertools


def debugging_singlet_direct_ci_method(h1e, h2e, norb, nelec):
    nalpha = nelec//2
    nbeta = nelec//2

    possiblestrings = list(itertools.combinations(range(norb), nalpha))
    possiblestringsarray = [list(x) for x in possiblestrings]
    numb_alpha_dets = len(possiblestrings)

    def nswaps(det):
        swaps = 0
        thisdet = deepcopy(det)
        sorteddet = sorted(thisdet)
        issorted = False
        while not issorted:
            if thisdet == sorteddet:
                issorted = True
            for i in range(len(thisdet)-1):
                if thisdet[i] > thisdet[i+1]:
                    deti = thisdet[i]
                    detnext = thisdet[i+1]
                    thisdet[i] = detnext
                    thisdet[i+1] = deti
                    swaps += 1
        return swaps

    def f(input_index, mu, nu):
        det = deepcopy(possiblestringsarray[input_index])
        if nu not in det:
            return None
        else:
            det.remove(nu)
            if mu in det:
                return None
            else:
                det.append(mu)
                det = sorted(det)
                det = tuple(det)
                output_index = possiblestrings.index(det)
                return output_index
    
    def sigma(input_index, mu, nu):
        det = deepcopy(possiblestringsarray[input_index])
        if nu not in det:
            return None
        else:
            det.remove(nu)
            if mu in det:
                return None
            else:
                det.append(mu)
                swaps = nswaps(det)
                return swaps
    


    # This is defined for spin-free; but now we need separate for alpha vs. for beta.
        
    def big_sigma(input_index, mu, nu):
        alpha_index =  input_index % numb_alpha_dets
        beta_index = (input_index - alpha_index) // numb_alpha_dets

        if mu < norb and nu < norb:
            #we act only on alpha
            return sigma(alpha_index, mu, nu)

        elif mu >= norb and nu >= norb:
            # do something
            return sigma(beta_index, mu % norb, nu % norb)

        else:
            return None


    def big_f(input_index, mu, nu):
        alpha_index = input_index % numb_alpha_dets
        beta_index = (input_index - alpha_index) // numb_alpha_dets

        if mu < norb and nu < norb:
            # we act only on alpha
            new_alpha_index = f(alpha_index, mu, nu)
            if new_alpha_index is None:
                return None
            else:
                output_index = beta_index * numb_alpha_dets + new_alpha_index
                return output_index

        elif mu >= norb and nu >= norb:
            # we act only on beta

            new_beta_index = f(beta_index, mu % norb, nu % norb)
            if new_beta_index is None:
                return None
            else:
                output_index = new_beta_index * numb_alpha_dets + alpha_index
                return output_index

        else:
            return None


    # now what happens when we act H onto every SD?

    hamiltonian = np.zeros((len(possiblestrings)**2, len(possiblestrings)**2))


    gijjk = np.einsum('ijjk->ik', h2e)

    def one_electron_term():
        for i in range(numb_alpha_dets**2):
            for j in range(norb*2):
                for k in range(norb*2):
                    if big_sigma(i, j, k) is not None:
                        # hamiltonian[i][big_f(i,j,k)] += (h1e[j%norb, k%norb] - 0.5*gijjk[j%norb, k%norb]) * (-1)**(big_sigma(i,j,k))
                        hamiltonian[i][big_f(i,j,k)] += (h1e[j%norb, k%norb]) * (-1)**(big_sigma(i,j,k))
    

    # def two_electron_coulomb():
    #     for term in range(numb_alpha_dets**2):
    #         for i in range(norb*2):
    #             for j in range(norb*2):
    #                 f_inter = big_f(term, i, j)
    #                 sig_inter = big_sigma(term, i, j)
    #                 if sig_inter is not None:
    #                     for k in range(norb*2):
    #                         for l in range(norb*2):

    #                             f_final = big_f(f_inter, k, l)
    #                             sig_final = big_sigma(f_inter, k, l)

    #                             if sig_final is not None:
    #                                 # hamiltonian[term][f_final] += 0.5*h2e[i%norb, j%norb, k%norb, l%norb] * (-1)**(sig_inter + sig_final)
    #                                 hamiltonian[term][f_final] += 0.0

    one_electron_term()
    # two_electron_coulomb()

    return hamiltonian   

debug_ham_dir = singlet_direct_ci_method(
    h1e=np.load("h1e.npy"),
    h2e=np.load("h2e.npy"),
    norb=6,
    nelec=6
)

eigvals_direct = np.linalg.eigh(debug_ham_dir)[0]
print(eigvals_direct)

[-4.34321743 -4.33031592 -4.22504842 -4.20932702 -4.19708957 -4.17335095
 -4.15183897 -4.15027799 -4.12785092 -4.11834982 -4.11429289 -4.0998278
 -4.0832055  -4.06531337 -4.05604433 -4.04518844 -4.03962505 -4.03317915
 -4.01923198 -4.01733628 -4.00517404 -4.00046274 -3.98638762 -3.97868783
 -3.97820893 -3.97095303 -3.95278117 -3.9501999  -3.93680307 -3.92560642
 -3.9124052  -3.90980824 -3.9036209  -3.88895061 -3.88426553 -3.88423608
 -3.87676485 -3.86345606 -3.85966158 -3.84976798 -3.84379892 -3.84102559
 -3.83202906 -3.81995069 -3.81632744 -3.81034228 -3.80879177 -3.79743885
 -3.78496792 -3.78049181 -3.77811735 -3.77461328 -3.76762894 -3.76537135
 -3.75883571 -3.75452538 -3.74705589 -3.73296572 -3.72925838 -3.7238987
 -3.71885586 -3.70382288 -3.70301187 -3.70098267 -3.69615038 -3.68946627
 -3.68572176 -3.6826509  -3.67204648 -3.66372501 -3.6525067  -3.64464246
 -3.64336514 -3.63903302 -3.63575527 -3.63187214 -3.62550834 -3.61972986
 -3.61473005 -3.60735444 -3.60634393 -3.60019387 -3.5

In [94]:
hammy_debug_fci = debugging_rudimentary_fci_matrix(
    h1e = np.load("h1e.npy"),
    h2e = np.load("h2e.npy"),
    norb = 6,
    nelec = 6
)

eigvals_debug_fci = np.linalg.eigh(hammy_debug_fci)[0]
print(eigvals_debug_fci)

[-12.74941451 -12.4797119  -12.4797119  -12.4797119  -12.4797119
 -12.24800325 -12.24800325 -12.24800325 -12.24800325 -12.21000928
 -12.16093448 -12.16093448 -12.16093448 -12.16093448 -12.05613086
 -12.05613086 -12.05613086 -12.05613086 -11.99446467 -11.99446467
 -11.99446467 -11.99446467 -11.97830064 -11.97830064 -11.97830064
 -11.97830064 -11.92922583 -11.92922583 -11.92922583 -11.92922583
 -11.89123186 -11.89123186 -11.89123186 -11.89123186 -11.78642825
 -11.78642825 -11.78642825 -11.78642825 -11.76275602 -11.76275602
 -11.76275602 -11.76275602 -11.74659199 -11.73735344 -11.73735344
 -11.73735344 -11.73735344 -11.72476205 -11.72476205 -11.72476205
 -11.72476205 -11.65952322 -11.65952322 -11.65952322 -11.65952322
 -11.65952322 -11.65952322 -11.65952322 -11.65952322 -11.65952322
 -11.65952322 -11.65952322 -11.65952322 -11.65952322 -11.65952322
 -11.65952322 -11.65952322 -11.57245444 -11.57088363 -11.57088363
 -11.57088363 -11.57088363 -11.5547196  -11.5547196  -11.5547196
 -11.5547196

In [96]:
new_ham_fci_zeros = rudimentary_fci_matrix(
    h1e = np.load("h1e.npy"),
    h2e = np.zeros((6, 6, 6, 6)),
    norb = 6,
    nelec = 6
)

In [97]:
print(np.linalg.eigh(new_ham_fci_zeros)[0])

[-12.74941451 -12.4797119  -12.4797119  -12.4797119  -12.4797119
 -12.24800325 -12.24800325 -12.24800325 -12.24800325 -12.21000928
 -12.16093448 -12.16093448 -12.16093448 -12.16093448 -12.05613086
 -12.05613086 -12.05613086 -12.05613086 -11.99446467 -11.99446467
 -11.99446467 -11.99446467 -11.97830064 -11.97830064 -11.97830064
 -11.97830064 -11.92922583 -11.92922583 -11.92922583 -11.92922583
 -11.89123186 -11.89123186 -11.89123186 -11.89123186 -11.78642825
 -11.78642825 -11.78642825 -11.78642825 -11.76275602 -11.76275602
 -11.76275602 -11.76275602 -11.74659199 -11.73735344 -11.73735344
 -11.73735344 -11.73735344 -11.72476205 -11.72476205 -11.72476205
 -11.72476205 -11.65952322 -11.65952322 -11.65952322 -11.65952322
 -11.65952322 -11.65952322 -11.65952322 -11.65952322 -11.65952322
 -11.65952322 -11.65952322 -11.65952322 -11.65952322 -11.65952322
 -11.65952322 -11.65952322 -11.57245444 -11.57088363 -11.57088363
 -11.57088363 -11.57088363 -11.5547196  -11.5547196  -11.5547196
 -11.5547196